In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [3]:
import torch

!pip uninstall torch-geometric --y
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Found existing installation: torch-geometric 2.7.0
Uninstalling torch-geometric-2.7.0:
  Successfully uninstalled torch-geometric-2.7.0
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-5kfa61l7
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-5kfa61l7
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 56d53d03a7326c7882d33345a759df1b02bcb4f2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.7.0-py3-none-any.whl size=1136511 sha256=55de5ae379756334835e185caa381ca3e1b777eba52bbcaba0689054a0b346f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-gfm3fumx/wheels/d3/78/eb/9e26525b948d19533f1688fb6c209cec8a0ba793d39b49ae8f
Successfully built torch-geometric


In [4]:
import json
import os
import random
import numpy as np
import pandas as pd
import cv2
from IPython.display import HTML
from base64 import b64encode
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch.nn.modules.normalization import LayerNorm
import torch.nn.init as init
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from collections import Counter

In [5]:
n_part = 128
n_graph_out = 16
n_rnn = 128
n_rnn_out = 15
frame_nos = 75

In [ ]:
def set_seeds(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seeds()

def read_json_file(file_path):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        return data
    except json.JSONDecodeError:
        print(f"Error reading JSON file: {file_path}")
        return None

def calculate_skip_and_keep(frame_rate, target_fps):
    """
    Calculate the number of frames to skip and the number of frames to keep based on the frame rate and target fps.

    Parameters:
    frame_rate (int): The original frame rate of the video.
    target_fps (int): The target frame rate after downsampling.

    Returns:
    skip_every_n (int): The number of frames to skip in each block.
    keep_every_m (int): The number of frames to keep in each block.
    """
    # Calculate the total frames in a block (skip + keep)
    total_frames_in_block = frame_rate

    # Calculate the number of frames to keep in each block
    keep_every_m = int((target_fps / frame_rate) * total_frames_in_block)

    # Calculate the number of frames to skip in each block
    skip_every_n = total_frames_in_block - keep_every_m

    return skip_every_n, keep_every_m

def process_openpose_output(directory, batch_size, frame_rate=25, target_fps=25, skip_every_n=0, keep_every_m=0, ignore_first_n=0):
    json_files = sorted([f for f in os.listdir(directory) if f.endswith('.json')])
    num_files = len(json_files)

    if skip_every_n > 0 and keep_every_m > 0:
        step_pattern = [(i % (skip_every_n + keep_every_m) < keep_every_m) for i in range(frame_rate)]
    else:
        step_pattern = [(i % (frame_rate // target_fps) == 0) for i in range(frame_rate)]

    selected_files = [f for i, f in enumerate(json_files[ignore_first_n:]) if step_pattern[i % frame_rate]]

    if len(selected_files) < (batch_size - 1) + 1:
        raise ValueError(f"Not enough files in the directory after applying the skip pattern. Found {len(selected_files)} files.")

    batches = [selected_files[i:i + batch_size] for i in range(0, len(selected_files), batch_size)]

    if len(batches[-1]) < batch_size:
        batches = batches[:-1]

    all_batches_data = []
    for batch in batches:
        batch_data = []
        for json_file in batch:
            file_path = os.path.join(directory, json_file)
            data = read_json_file(file_path)
            if data:
                batch_data.append(data)
        if batch_data:
            # Include the directory path with each batch
            all_batches_data.append((batch_data, batch, directory))

    return all_batches_data

frame_rate = 25  # Original FPS of the data
target_fps = 15  # Desired FPS

# Calculate skip and keep parameters dynamically
skip_every_n, keep_every_m = calculate_skip_and_keep(frame_rate, target_fps)

batch_size = 75  # Number of frames per batch

def flip_keypoints_horizontally(keypoints, image_width=1):
    flipped_keypoints = keypoints.copy()
    flip_pairs = [
        (2, 5), (3, 6), (4, 7), (9, 12), (10, 13), (11, 14), (15, 16), (17, 18), (19, 22), (20, 23), (21, 24)
    ]

    for i in range(len(keypoints) // 3):
        if keypoints[i * 3] >= 0:
            flipped_keypoints[i * 3] = image_width - keypoints[i * 3]
        else:
            flipped_keypoints[i * 3] = keypoints[i * 3]

    for (i, j) in flip_pairs:
        flipped_keypoints[i * 3], flipped_keypoints[j * 3] = flipped_keypoints[j * 3], flipped_keypoints[i * 3]
        flipped_keypoints[i * 3 + 1], flipped_keypoints[j * 3 + 1] = flipped_keypoints[j * 3 + 1], flipped_keypoints[i * 3 + 1]
        flipped_keypoints[i * 3 + 2], flipped_keypoints[j * 3 + 2] = flipped_keypoints[j * 3 + 2], flipped_keypoints[i * 3 + 2]

    return flipped_keypoints

def create_matrix(data, selected_files, augment=False):
    num_frames = len(selected_files)
    matrix = [[None] * num_frames for _ in range(len(body_parts))]

    for frame_number, (frame_data, json_file) in enumerate(zip(data, selected_files)):
        if frame_data['people']:
            keypoints = frame_data['people'][0]['pose_keypoints_2d']

            if augment:
                keypoints = flip_keypoints_horizontally(keypoints)

            if len(keypoints) == len(body_parts) * 1.5:
                for i in range(len(body_parts) // 2):
                    x, y, confidence = keypoints[i * 3:(i + 1) * 3]

                    if confidence > 0:
                        matrix[i * 2][frame_number] = x
                        matrix[i * 2 + 1][frame_number] = y
                    else:
                        matrix[i * 2][frame_number] = -1
                        matrix[i * 2 + 1][frame_number] = -1
            else:
                for i in range(len(body_parts) // 2):
                    matrix[i * 2][frame_number] = None
                    matrix[i * 2 + 1][frame_number] = None
        else:
            for i in range(len(body_parts) // 2):
                matrix[i * 2][frame_number] = None
                matrix[i * 2 + 1][frame_number] = None

    column_labels = [f'Frame_{os.path.splitext(f)[0]}' for f in selected_files]
    df = pd.DataFrame(matrix, index=row_labels, columns=column_labels)
    df.fillna(-1, inplace=True)

    return df

body_parts = [
    "x_Nose", "y_Nose", "x_Neck", "y_Neck", "x_RShoulder", "y_RShoulder", "x_RElbow", "y_RElbow", "x_RWrist", "y_RWrist",
    "x_LShoulder", "y_LShoulder", "x_LElbow", "y_LElbow", "x_LWrist", "y_LWrist", "x_MidHip", "y_MidHip", "x_RHip", "y_RHip",
    "x_RKnee", "y_RKnee", "x_RAnkle", "y_RAnkle", "x_LHip", "y_LHip", "x_LKnee", "y_LKnee", "x_LAnkle", "y_LAnkle",
    "x_REye", "y_REye", "x_LEye", "y_LEye", "x_REar", "y_REar", "x_LEar", "y_LEar", "x_LBigToe", "y_LBigToe",
    "x_LSmallToe", "y_LSmallToe", "x_LHeel", "y_LHeel", "x_RBigToe", "y_RBigToe", "x_RSmallToe", "y_RSmallToe", "x_RHeel", "y_RHeel"
]

row_labels = body_parts

directories = [
    '/content/drive/My Drive/patient_openpose/00001',
    '/content/drive/My Drive/patient_openpose/00002_s',
    '/content/drive/My Drive/patient_openpose/00003_s',
    '/content/drive/My Drive/patient_openpose/00004',
    '/content/drive/My Drive/patient_openpose/00005',
    '/content/drive/My Drive/patient_openpose/00006',
    '/content/drive/My Drive/patient_openpose/00007',
    '/content/drive/My Drive/patient_openpose/00008',
    '/content/drive/My Drive/patient_openpose/00009',
    '/content/drive/My Drive/patient_openpose/00010_s',
    '/content/drive/My Drive/patient_openpose/00011',
    '/content/drive/My Drive/patient_openpose/00012',
    '/content/drive/My Drive/patient_openpose/00013',
    '/content/drive/My Drive/patient_openpose/00014',
    '/content/drive/My Drive/patient_openpose/00016',
    '/content/drive/My Drive/patient_openpose/00017',
    '/content/drive/My Drive/patient_openpose/00018_s',
    '/content/drive/My Drive/patient_openpose/00019',
    '/content/drive/My Drive/patient_openpose/00020',
    '/content/drive/My Drive/patient_openpose/00021',
    '/content/drive/My Drive/patient_openpose/00022',
    '/content/drive/My Drive/patient_openpose/00024',
    '/content/drive/My Drive/patient_openpose/00025',
    '/content/drive/My Drive/patient_openpose/00026',
    '/content/drive/My Drive/patient_openpose/00027',
    '/content/drive/My Drive/patient_openpose/00028_s',
    '/content/drive/My Drive/patient_openpose/00029',
    '/content/drive/My Drive/patient_openpose/00030',
    '/content/drive/My Drive/patient_openpose/00031',
    '/content/drive/My Drive/patient_openpose/00032',
    '/content/drive/My Drive/patient_openpose/00033',
    '/content/drive/My Drive/patient_openpose/00034',
    '/content/drive/My Drive/patient_openpose/00035',
    '/content/drive/My Drive/patient_openpose/00036',
    '/content/drive/My Drive/patient_openpose/00037',
    '/content/drive/My Drive/patient_openpose/00038',
    '/content/drive/My Drive/patient_openpose/00039',
    '/content/drive/My Drive/patient_openpose/00041',
    '/content/drive/My Drive/patient_openpose/00042',
    '/content/drive/My Drive/patient_openpose/00044',
    '/content/drive/My Drive/patient_openpose/00045',
    '/content/drive/My Drive/patient_openpose/00046',
    '/content/drive/My Drive/patient_openpose/00047',
    '/content/drive/My Drive/patient_openpose/00048',
    '/content/drive/My Drive/patient_openpose/00049',
    '/content/drive/My Drive/patient_openpose/00050',
    '/content/drive/My Drive/patient_openpose/00051',
    '/content/drive/My Drive/patient_openpose/00052',
    '/content/drive/My Drive/patient_openpose/00053',
    '/content/drive/My Drive/patient_openpose/00054',
]
frame_nos = 75
target = torch.tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
                       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1], dtype=torch.long)

original_distribution = Counter(target.tolist())
print("Original label distribution:", original_distribution)

minority_class_indices = [i for i, label in enumerate(target) if label == 1]
majority_class_indices = [i for i, label in enumerate(target) if label == 0]

np.random.seed(42)
bootstrapped_minority_indices = np.random.choice(minority_class_indices, size=len(majority_class_indices), replace=True)

balanced_indices = np.concatenate([majority_class_indices, bootstrapped_minority_indices])

directories_balanced = [directories[i] for i in balanced_indices]
target_balanced = target[balanced_indices]

train_dirs, temp_dirs, train_labels, temp_labels = train_test_split(
    directories_balanced, target_balanced, test_size=0.2, random_state=42, stratify=target_balanced
)
val_dirs, test_dirs, val_labels, test_labels = train_test_split(
    temp_dirs, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

train_label_counts = Counter([label.item() for label in train_labels])
val_label_counts = Counter([label.item() for label in val_labels])
test_label_counts = Counter([label.item() for label in test_labels])

print("Training label distribution:", train_label_counts)
print("Validation label distribution:", val_label_counts)
print("Testing label distribution:", test_label_counts)

def process_and_augment_directories_with_labels(directories, labels, frame_rate=25, target_fps=25, batch_size=75):
    matrices = []
    batch_labels = []
    batch_directories = []
    skip_every_n, keep_every_m = calculate_skip_and_keep(frame_rate, target_fps)

    for directory, label in zip(directories, labels):
        try:
            batches_data = process_openpose_output(directory, batch_size=batch_size, frame_rate=frame_rate, target_fps=target_fps, skip_every_n=skip_every_n, keep_every_m=keep_every_m)
            for batch_data, batch_files, batch_directory in batches_data:
                df = create_matrix(batch_data, batch_files)
                df_aug = create_matrix(batch_data, batch_files, augment=True)
                matrices.append(df)
                matrices.append(df_aug)
                batch_labels.extend([label, label])
                batch_directories.extend([batch_directory, batch_directory])
        except ValueError as e:
            print(f"Skipping directory {directory} due to insufficient frames: {e}")
            continue
    return matrices, batch_labels, batch_directories

train_matrices, train_batch_labels, train_batch_dirs = process_and_augment_directories_with_labels(train_dirs, train_labels, frame_rate=frame_rate, target_fps=target_fps, batch_size=batch_size)
val_matrices, val_batch_labels, val_batch_dirs = process_and_augment_directories_with_labels(val_dirs, val_labels, frame_rate=frame_rate, target_fps=target_fps, batch_size=batch_size)
test_matrices, test_batch_labels, test_batch_dirs = process_and_augment_directories_with_labels(test_dirs, test_labels, frame_rate=frame_rate, target_fps=target_fps, batch_size=batch_size)

print("Directories for training batches:", train_batch_dirs)
print("Directories for validation batches:", val_batch_dirs)
print("Directories for test batches:", test_batch_dirs)

train_batch_labels = torch.tensor(train_batch_labels, dtype=torch.long)
val_batch_labels = torch.tensor(val_batch_labels, dtype=torch.long)
test_batch_labels = torch.tensor(test_batch_labels, dtype=torch.long)
train_tensor = np.stack([df.values for df in train_matrices], axis=0)
val_tensor = np.stack([df.values for df in val_matrices], axis=0)
test_tensor = np.stack([df.values for df in test_matrices], axis=0)

print("Train tensor shape:", train_tensor.shape)
print("Train labels shape:", train_batch_labels.shape)
print("Validation tensor shape:", val_tensor.shape)
print("Validation labels shape:", val_batch_labels.shape)
print("Test tensor shape:", test_tensor.shape)
print("Test labels shape:", test_batch_labels.shape)


In [6]:
# Load the train, validation, and test tensors
train_tensor = np.load('/content/drive/My Drive/patient_openpose/train_tensor.npy')
val_tensor = np.load('/content/drive/My Drive/patient_openpose/val_tensor.npy')
test_tensor = np.load('/content/drive/My Drive/patient_openpose/test_tensor.npy')

# Load the corresponding labels
train_labels = np.load('/content/drive/My Drive/patient_openpose/train_labels.npy')
val_labels = np.load('/content/drive/My Drive/patient_openpose/val_labels.npy')
test_labels = np.load('/content/drive/My Drive/patient_openpose/test_labels.npy')

# If you need them as PyTorch tensors, convert them
train_tensor = torch.tensor(train_tensor)
val_tensor = torch.tensor(val_tensor)
test_tensor = torch.tensor(test_tensor)

train_labels = torch.tensor(train_labels, dtype=torch.long)
val_labels = torch.tensor(val_labels, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.long)

# Now you can use train_tensor, val_tensor, test_tensor, train_labels, val_labels, test_labels in your code
print("Train tensor shape:", train_tensor.shape)
print("Train labels shape:", train_labels.shape)
print("Validation tensor shape:", val_tensor.shape)
print("Validation labels shape:", val_labels.shape)
print("Test tensor shape:", test_tensor.shape)
print("Test labels shape:", test_labels.shape)


Train tensor shape: torch.Size([318, 50, 75])
Train labels shape: torch.Size([318])
Validation tensor shape: torch.Size([32, 50, 75])
Validation labels shape: torch.Size([32])
Test tensor shape: torch.Size([88, 50, 75])
Test labels shape: torch.Size([88])


In [7]:
left_arm_train_np = np.concatenate((train_tensor[:, 2:4, :], train_tensor[:, 10:16, :]), axis=1)
print(left_arm_train_np.shape)
right_arm_train_np = np.concatenate((train_tensor[:, 2:4, :], train_tensor[:, 4:10, :]), axis=1)
print(right_arm_train_np.shape)
left_leg_train_np = np.concatenate((train_tensor[:, 16:18, :], train_tensor[:, 24:30, :], train_tensor[:, 38:44, :]), axis=1)
print(left_leg_train_np.shape)
right_leg_train_np = np.concatenate((train_tensor[:, 16:24, :], train_tensor[:, 44:50, :]), axis=1)
print(right_leg_train_np.shape)
torso_train_np = np.concatenate((train_tensor[:, 0:4, :], train_tensor[:, 30:38, :]), axis=1)
print(torso_train_np.shape)

left_arm_test_np = np.concatenate((test_tensor[:, 2:4, :], test_tensor[:, 10:16, :]), axis=1)
print(left_arm_test_np.shape)
right_arm_test_np = np.concatenate((test_tensor[:, 2:4, :], test_tensor[:, 4:10, :]), axis=1)
print(right_arm_test_np.shape)
left_leg_test_np = np.concatenate((test_tensor[:, 16:18, :], test_tensor[:, 24:30, :], test_tensor[:, 38:44, :]), axis=1)
print(left_leg_test_np.shape)
right_leg_test_np = np.concatenate((test_tensor[:, 16:24, :], test_tensor[:, 44:50, :]), axis=1)
print(right_leg_test_np.shape)
torso_test_np = np.concatenate((test_tensor[:, 0:4, :], test_tensor[:, 30:38, :]), axis=1)
print(torso_test_np.shape)

left_arm_val_np = np.concatenate((val_tensor[:, 2:4, :], val_tensor[:, 10:16, :]), axis=1)
print(left_arm_val_np.shape)
right_arm_val_np = np.concatenate((val_tensor[:, 2:4, :], val_tensor[:, 4:10, :]), axis=1)
print(right_arm_val_np.shape)
left_leg_val_np = np.concatenate((val_tensor[:, 16:18, :], val_tensor[:, 24:30, :], val_tensor[:, 38:44, :]), axis=1)
print(left_leg_val_np.shape)
right_leg_val_np = np.concatenate((val_tensor[:, 16:24, :], val_tensor[:, 44:50, :]), axis=1)
print(right_leg_val_np.shape)
torso_val_np = np.concatenate((val_tensor[:, 0:4, :], val_tensor[:, 30:38, :]), axis=1)
print(torso_val_np.shape)

(318, 8, 75)
(318, 8, 75)
(318, 14, 75)
(318, 14, 75)
(318, 12, 75)
(88, 8, 75)
(88, 8, 75)
(88, 14, 75)
(88, 14, 75)
(88, 12, 75)
(32, 8, 75)
(32, 8, 75)
(32, 14, 75)
(32, 14, 75)
(32, 12, 75)


In [8]:
left_arm_train_tensor = torch.from_numpy(left_arm_train_np).float()
right_arm_train_tensor = torch.from_numpy(right_arm_train_np).float()
left_leg_train_tensor = torch.from_numpy(left_leg_train_np).float()
right_leg_train_tensor = torch.from_numpy(right_leg_train_np).float()
torso_train_tensor = torch.from_numpy(torso_train_np).float()

left_arm_test_tensor = torch.from_numpy(left_arm_test_np).float()
right_arm_test_tensor = torch.from_numpy(right_arm_test_np).float()
left_leg_test_tensor = torch.from_numpy(left_leg_test_np).float()
right_leg_test_tensor = torch.from_numpy(right_leg_test_np).float()
torso_test_tensor = torch.from_numpy(torso_test_np).float()

left_arm_val_tensor = torch.from_numpy(left_arm_val_np).float()
right_arm_val_tensor = torch.from_numpy(right_arm_val_np).float()
left_leg_val_tensor = torch.from_numpy(left_leg_val_np).float()
right_leg_val_tensor = torch.from_numpy(right_leg_val_np).float()
torso_val_tensor = torch.from_numpy(torso_val_np).float()

In [9]:
rnn_sequence_train = np.concatenate((train_tensor[:, 2:4, :], right_leg_train_np, train_tensor[:, 20:22, :], train_tensor[:, 18:20, :], left_leg_train_np, train_tensor[:, 26:28, :], train_tensor[:, 24:26, :], train_tensor[:, 16:18, :], torso_train_np, right_arm_train_np, train_tensor[:, 6:8, :], train_tensor[:, 4:6, :], left_arm_train_np, train_tensor[:, 12:14, :], train_tensor[:, 10:12, :], train_tensor[:, 2:4, :]), axis=1)
rnn_sequence_train = np.array(rnn_sequence_train, dtype=float)
rnn_sequence_train_tensor = torch.from_numpy(rnn_sequence_train).float()
reshaped_rnn_train_tensor = rnn_sequence_train_tensor.permute(0, 2, 1)
print(reshaped_rnn_train_tensor.shape)

rnn_sequence_test = np.concatenate((test_tensor[:, 2:4, :], right_leg_test_np, test_tensor[:, 20:22, :], test_tensor[:, 18:20, :], left_leg_test_np, test_tensor[:, 26:28, :], test_tensor[:, 24:26, :], test_tensor[:, 16:18, :], torso_test_np, right_arm_test_np, test_tensor[:, 6:8, :], test_tensor[:, 4:6, :], left_arm_test_np, test_tensor[:, 12:14, :], test_tensor[:, 10:12, :], test_tensor[:, 2:4, :]), axis=1)
rnn_sequence_test = np.array(rnn_sequence_test, dtype=float)
rnn_sequence_test_tensor = torch.from_numpy(rnn_sequence_test).float()
reshaped_rnn_test_tensor = rnn_sequence_test_tensor.permute(0, 2, 1)
print(reshaped_rnn_test_tensor.shape)

rnn_sequence_val = np.concatenate((val_tensor[:, 2:4, :], right_leg_val_np, val_tensor[:, 20:22, :], val_tensor[:, 18:20, :], left_leg_val_np, val_tensor[:, 26:28, :], val_tensor[:, 24:26, :], val_tensor[:, 16:18, :], torso_val_np, right_arm_val_np, val_tensor[:, 6:8, :], val_tensor[:, 4:6, :], left_arm_val_np, val_tensor[:, 12:14, :], val_tensor[:, 10:12, :], val_tensor[:, 2:4, :]), axis=1)
rnn_sequence_val = np.array(rnn_sequence_val, dtype=float)
rnn_sequence_val_tensor = torch.from_numpy(rnn_sequence_val).float()
reshaped_rnn_val_tensor = rnn_sequence_val_tensor.permute(0, 2, 1)
print(reshaped_rnn_val_tensor.shape)


torch.Size([318, 75, 78])
torch.Size([88, 75, 78])
torch.Size([32, 75, 78])


In [10]:
class SelfAttention(nn.Module):
    def __init__(self, num_parts, input_dim):
        super(SelfAttention, self).__init__()
        self.num_parts = num_parts
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)
        self.softmax = nn.Softmax(dim=2)  # Apply softmax over the parts

    def forward(self, x):
        # x is of shape [batch_size, num_parts, input_dim] (e.g., [318, 5, 128])
        queries = self.query(x)  # Shape: [batch_size, num_parts, input_dim]
        keys = self.key(x)        # Shape: [batch_size, num_parts, input_dim]
        values = self.value(x)    # Shape: [batch_size, num_parts, input_dim]

        # Compute attention scores across the body parts
        scores = torch.bmm(queries, keys.transpose(1, 2)) / (self.num_parts ** 0.5)  # Shape: [batch_size, num_parts, num_parts]
        attention_weights = self.softmax(scores)  # Shape: [batch_size, num_parts, num_parts]

        # Compute weighted sum of values based on attention weights
        weighted_values = torch.bmm(attention_weights, values)  # Shape: [batch_size, num_parts, input_dim]

        return weighted_values, scores, attention_weights


In [11]:
num_nodes = 5
edges = []
for i in range(num_nodes):
    for j in range(num_nodes):
        if i != j:
            edges.append([i, j])

edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
print(edge_index)

tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
        [1, 2, 3, 4, 0, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 4, 0, 1, 2, 3]])


In [12]:
class ResidualBlock(nn.Module):
    def __init__(self, input_dim, output_dim, leaky_relu_alpha):
        super(ResidualBlock, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.bn = nn.BatchNorm1d(output_dim)
        self.leaky_relu = nn.LeakyReLU(negative_slope=leaky_relu_alpha)
        self.dropout = nn.Dropout(p=dropout_prob)

        if input_dim != output_dim:
            self.residual_connection = nn.Linear(input_dim, output_dim)
        else:
            self.residual_connection = None

    def forward(self, x):
        residual = x
        out = self.fc(x)
        out = self.bn(out)
        out = self.leaky_relu(out)
        out = self.dropout(out)

        if self.residual_connection is not None:
            residual = self.residual_connection(residual)

        out += residual
        return out

class SpatialNetwork(nn.Module):
    def __init__(self, n_part, n_graph_out, frame_nos, leaky_relu_alpha):
        super(SpatialNetwork, self).__init__()

        self.n_part = n_part
        self.n_graph_out = n_graph_out
        self.frame_nos = frame_nos

        self.body_parts_fc = nn.ModuleDict({
            'arms': nn.Linear(8, n_part),
            'legs': nn.Linear(14, n_part),
            'torso': nn.Linear(12, n_part),
        })

        self.self_attn = SelfAttention(num_parts=5, input_dim=n_part)
        self.gcn1 = GCNConv(in_channels=n_part, out_channels=n_graph_out, aggr='max', bias=True)
        self.bn_gcn1 = nn.BatchNorm1d(n_graph_out)
        self.fc6 = ResidualBlock(n_graph_out * frame_nos, 512, leaky_relu_alpha)
        self.fc7 = ResidualBlock(512, 256, leaky_relu_alpha)

        self.training_attention_scores = []  # Store only the last epoch's training scores
        self.training_attention_weights = []

    def reset_attention_scores(self):
        self.training_attention_scores = []
        self.training_attention_weights = []

    def forward_body_part(self, x, part):
        x = self.body_parts_fc[part](x)
        x = F.leaky_relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        return x

    def forward(self, left_arm, right_arm, left_leg, right_leg, torso, edge_index, is_training=True):
        batch_size, features, seq_len = left_arm.shape

        outputs = torch.zeros((batch_size, seq_len, self.n_graph_out), device=left_arm.device)

        for t in range(seq_len):
            left_arm_out = self.forward_body_part(left_arm[:, :, t], 'arms')
            right_arm_out = self.forward_body_part(right_arm[:, :, t], 'arms')
            left_leg_out = self.forward_body_part(left_leg[:, :, t], 'legs')
            right_leg_out = self.forward_body_part(right_leg[:, :, t], 'legs')
            torso_out = self.forward_body_part(torso[:, :, t], 'torso')

            concatenated = torch.stack((left_arm_out, right_arm_out, left_leg_out, right_leg_out, torso_out), dim=1)

            attn_output, attn_scores, attn_weights = self.self_attn(concatenated)

            if is_training:
                # Only accumulate attention scores during training
                self.training_attention_scores.append(attn_scores)
                self.training_attention_weights.append(attn_weights)

            batch_size, num_nodes, feature_dim = attn_output.shape
            attn_output = attn_output.reshape(batch_size * num_nodes, feature_dim)

            gcn_output = self.gcn1(attn_output, edge_index)
            gcn_output = self.bn_gcn1(gcn_output)
            gcn_output = F.leaky_relu(gcn_output)
            gcn_output = F.dropout(gcn_output, p=0.5, training=self.training)
            gcn_output = gcn_output.view(batch_size, num_nodes, -1)
            pooled_output = torch.max(gcn_output, dim=1)[0]

            outputs[:, t, :] = pooled_output

        flattened_outputs = outputs.view(batch_size, -1)
        fc6_output = self.fc6(flattened_outputs)
        fc7_output = self.fc7(fc6_output)

        return fc7_output

    def get_last_epoch_attention_scores(self):
        # Concatenate the stored attention scores from the last epoch's training phase
        if not self.training_attention_scores:
            raise ValueError("No training attention scores stored. Ensure that the model was trained and attention scores were captured.")

        attention_scores = torch.cat(self.training_attention_scores, dim=0)

        # Expecting training batch_size = 318 and seq_len = 75
        return attention_scores.view(-1, 75, 5, 5).cpu().detach().numpy()

    def get_last_epoch_attention_weights(self):
        # Concatenate the stored attention weights from the last epoch's training phase
        if not self.training_attention_weights:
            raise ValueError("No training attention weights stored. Ensure that the model was trained and attention scores were captured.")

        attention_weights = torch.cat(self.training_attention_weights, dim=0)

        return attention_weights.view(-1, 75, 5, 5).cpu().detach().numpy()

In [13]:
class TemporalNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout_prob, leaky_relu_alpha):
        super(TemporalNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout_prob)

        self.self_attn = SelfAttention(num_parts=75, input_dim=hidden_size * 2)

        self.dropout = nn.Dropout(dropout_prob)

        self.fc1 = nn.Linear(hidden_size * 2, output_size * input_size)
        self.bn1 = nn.BatchNorm1d(output_size * input_size)
        self.fc2 = nn.Linear(output_size * input_size, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = ResidualBlock(512, 256, leaky_relu_alpha)  # Assuming ResidualBlock is defined elsewhere

        self.stored_attention_scores = []  # Store attention scores
        self.stored_attention_weights = []  # Store attention weights

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        # Get the output, attention scores, and weights from the SelfAttention module
        out, scores, weights = self.self_attn(out)

        # Store the scores and weights
        self.stored_attention_scores.append(scores)
        self.stored_attention_weights.append(weights)

        out = self.dropout(out)
        out = self.fc1(out[:, -1, :])  # Use the last time step's output
        out = out.view(out.size(0), -1)

        out = self.bn1(out)
        out = F.leaky_relu(out)

        out = self.fc2(out)
        out = self.bn2(out)
        out = F.leaky_relu(out)
        out = self.dropout(out)

        out = self.fc3(out)

        return out

    def clear_attention_weights(self):
        self.stored_attention_scores = []
        self.stored_attention_weights = []

    def get_attention_scores(self):
        return self.stored_attention_scores

    def get_attention_weights(self):
        return self.stored_attention_weights


In [14]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, input_dim, num_heads=4):
        super(MultiHeadSelfAttention, self).__init__()
        assert input_dim % num_heads == 0, "input_dim must be divisible by num_heads"
        self.num_heads = num_heads
        self.head_dim = input_dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.qkv_proj = nn.Linear(input_dim, input_dim * 3)
        self.fc_out = nn.Linear(input_dim, input_dim)

    def forward(self, x):
        batch_size, seq_length, input_dim = x.shape

        qkv = self.qkv_proj(x).reshape(batch_size, seq_length, self.num_heads, 3 * self.head_dim)
        q, k, v = qkv.chunk(3, dim=-1)

        q = q.permute(0, 2, 1, 3)
        k = k.permute(0, 2, 1, 3)
        v = v.permute(0, 2, 1, 3)

        attn_weights = (q @ k.transpose(-2, -1)) * self.scale
        attn_probs = F.softmax(attn_weights, dim=-1)

        attn_out = (attn_probs @ v).permute(0, 2, 1, 3).reshape(batch_size, seq_length, input_dim)
        return self.fc_out(attn_out)

class FinalConcatenation(nn.Module):
    def __init__(self, input_dim=512, output_dim=2, dropout_prob=0.5, num_heads=4):
        super(FinalConcatenation, self).__init__()
        self.attention = MultiHeadSelfAttention(input_dim=input_dim, num_heads=num_heads)
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(256, output_dim)

        self.layer_norm = LayerNorm(input_dim)
        self.residual_fc = nn.Linear(input_dim, input_dim)

    def forward(self, x):
        x = x.unsqueeze(1)  # [batch_size, seq_length, input_dim] -> [batch_size, 1, input_dim]

        res = x
        x = self.layer_norm(x)
        x = self.attention(x)
        x += self.residual_fc(res)

        x = x.squeeze(1)  # [batch_size, input_dim]

        x = self.fc1(x)  # [batch_size, 256]
        x = self.bn1(x)
        x = F.leaky_relu(x)
        x = self.dropout(x)
        x = self.fc2(x)  # [batch_size, output_dim]
        x = F.log_softmax(x, dim=1)
        return x


In [15]:
def initialize_weights(model):
    for name, param in model.named_parameters():
        if 'weight' in name:
            if param.dim() >= 2:
                nn.init.kaiming_normal_(param.data, nonlinearity='leaky_relu')
            else:
                param.data.fill_(1)
        elif 'bias' in name:
            param.data.fill_(0)

In [16]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

#got this from https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py

In [17]:
def custom_initialize_weights(model, gcn_constant=0.0001):
    for name, module in model.named_modules():
        if isinstance(module, GCNConv):
            init.constant_(module.lin.weight, gcn_constant)
            if module.lin.bias is not None:
                init.constant_(module.lin.bias, 0)
        elif isinstance(module, SelfAttention):
            init.kaiming_normal_(module.query.weight, nonlinearity='leaky_relu')
            init.kaiming_normal_(module.key.weight, nonlinearity='leaky_relu')
            init.kaiming_normal_(module.value.weight, nonlinearity='leaky_relu')
            if module.query.bias is not None:
                init.constant_(module.query.bias, 0)
            if module.key.bias is not None:
                init.constant_(module.key.bias, 0)
            if module.value.bias is not None:
                init.constant_(module.value.bias, 0)
        elif isinstance(module, nn.Linear):
            init.kaiming_normal_(module.weight, nonlinearity='leaky_relu')
            if module.bias is not None:
                init.constant_(module.bias, 0)
        elif isinstance(module, nn.BatchNorm1d):
            init.constant_(module.weight, 1)
            init.constant_(module.bias, 0)

In [18]:
leaky_relu_alpha = 0.01
input_size = 78
hidden_size = n_rnn
output_size = n_rnn_out
num_layers = 4
dropout_prob = 0.5

In [19]:
alpha=0.1
gamma=0
learning_rate=0.0001
weight_decay=0.001

In [20]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=2, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        log_probs = nn.functional.log_softmax(inputs, dim=1)
        probs = torch.exp(log_probs)
        targets_one_hot = torch.nn.functional.one_hot(targets, num_classes=log_probs.size(1)).float()

        focal_weight = torch.pow(1 - probs, self.gamma)
        focal_loss = -self.alpha * focal_weight * targets_one_hot * log_probs

        if self.reduction == 'mean':
            focal_loss = focal_loss.mean()
        elif self.reduction == 'sum':
            focal_loss = focal_loss.sum()

        return focal_loss


In [21]:
def cross_val_significance_test(dataset, labels, n_splits=5, batch_size=32, baseline=None, **train_params):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    performance_metrics = []

    for train_index, val_index in skf.split(labels, labels):
        train_loader = create_data_loader(dataset, train_index, batch_size)
        val_loader = create_data_loader(dataset, val_index, batch_size)

        # Train and evaluate the model on this fold
        _, accuracy, _, _ = train_model(**train_params, train_loader=train_loader, val_loader=val_loader)
        performance_metrics.append(accuracy)

    performance_metrics = np.array(performance_metrics)

    # Calculate confidence intervals
    mean_accuracy = np.mean(performance_metrics)
    ci_lower, ci_upper = bootstrap_confidence_interval(performance_metrics)

    if baseline is not None:
        t_stat, p_value = ttest_1samp(performance_metrics, baseline)
        print(f"Mean Performance: {mean_accuracy}, 95% CI: ({ci_lower}, {ci_upper}), Baseline: {baseline}, p-value: {p_value}")
        return mean_accuracy, ci_lower, ci_upper, p_value
    else:
        print(f"Mean Performance: {mean_accuracy}, 95% CI: ({ci_lower}, {ci_upper})")
        return mean_accuracy, ci_lower, ci_upper


In [29]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import StratifiedKFold
from scipy.stats import ttest_1samp
from torch.utils.data import DataLoader, Subset, TensorDataset

class_names = ['Class 0', 'Class 1']

def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

def bootstrap_confidence_interval(data, num_bootstrap_samples=1000, confidence_level=0.95):
    n = len(data)
    bootstrap_samples = np.random.choice(data, (num_bootstrap_samples, n), replace=True)
    bootstrap_means = np.mean(bootstrap_samples, axis=1)
    lower_bound = np.percentile(bootstrap_means, (1-confidence_level)/2 * 100)
    upper_bound = np.percentile(bootstrap_means, (1 + confidence_level)/2 * 100)

    return lower_bound, upper_bound

def create_data_loader(dataset, indices, batch_size):
    subset = Subset(dataset, indices)
    return DataLoader(subset, batch_size=batch_size, shuffle=True, drop_last=True)

def calculate_metrics(all_labels, all_preds):
    cm = confusion_matrix(all_labels, all_preds)
    tn, fp, fn, tp = cm.ravel()

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    # Manually calculate F1 score
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = sensitivity  # recall is the same as sensitivity
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    return accuracy, sensitivity, specificity, f1

def train_model(alpha, gamma, learning_rate, weight_decay, train_loader, val_loader, edge_index, class_names,
                spatial_net, temporal_net, final_model):
    def set_seeds(seed=42):
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    spatial_net = SpatialNetwork(n_part, n_graph_out, frame_nos, leaky_relu_alpha)
    custom_initialize_weights(spatial_net)

    temporal_net = TemporalNetwork(input_size, hidden_size, output_size, num_layers, dropout_prob, leaky_relu_alpha)
    initialize_weights(temporal_net)

    final_model = FinalConcatenation()
    initialize_weights(final_model)

    optimizer = optim.Adam(list(spatial_net.parameters()) + list(temporal_net.parameters()) + list(final_model.parameters()),
                           lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.001, step_size_up=2000, mode='triangular2')
    early_stopping = EarlyStopping(patience=10, verbose=True)

    num_epochs = 47
    criterion = FocalLoss(alpha=alpha, gamma=gamma)

    # Store metrics
    train_metrics = {'accuracy': [], 'sensitivity': [], 'specificity': [], 'f1': []}
    val_metrics = {'accuracy': [], 'sensitivity': [], 'specificity': [], 'f1': []}

    for epoch in range(num_epochs):
        temporal_net.clear_attention_weights()

        spatial_net.train()
        temporal_net.train()
        final_model.train()

        all_train_preds = []
        all_train_labels = []

        for batch in train_loader:
            left_arm_train_tensor, right_arm_train_tensor, left_leg_train_tensor, right_leg_train_tensor, torso_train_tensor, reshaped_rnn_train_tensor, train_labels = batch

            optimizer.zero_grad()

            output_spatial_train = spatial_net(left_arm_train_tensor, right_arm_train_tensor, left_leg_train_tensor,
                                               right_leg_train_tensor, torso_train_tensor, edge_index)
            output_temporal_train = temporal_net(reshaped_rnn_train_tensor)
            concatenated_input_train = torch.cat((output_spatial_train, output_temporal_train), dim=1)
            final_output_train = final_model(concatenated_input_train)

            loss = criterion(final_output_train, train_labels)
            loss.backward()

            optimizer.step()
            scheduler.step()

            _, train_preds = torch.max(final_output_train, 1)
            all_train_preds.extend(train_preds.cpu().numpy())
            all_train_labels.extend(train_labels.cpu().numpy())

            torch.nn.utils.clip_grad_norm_(spatial_net.parameters(), max_norm=1.0)
            torch.nn.utils.clip_grad_norm_(temporal_net.parameters(), max_norm=1.0)
            torch.nn.utils.clip_grad_norm_(final_model.parameters(), max_norm=1.0)

        # Calculate metrics for training
        train_accuracy, train_sensitivity, train_specificity, train_f1 = calculate_metrics(all_train_labels, all_train_preds)
        train_metrics['accuracy'].append(train_accuracy)
        train_metrics['sensitivity'].append(train_sensitivity)
        train_metrics['specificity'].append(train_specificity)
        train_metrics['f1'].append(train_f1)

        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item()}, Training Accuracy: {train_accuracy:.4f}, Training Sensitivity: {train_sensitivity:.4f}, Training Specificity: {train_specificity:.4f}, Training F1: {train_f1:.4f}")

        spatial_net.eval()
        temporal_net.eval()
        final_model.eval()

        all_val_preds = []
        all_val_labels = []
        val_loss = 0.0

        with torch.no_grad():
            for batch in val_loader:
                left_arm_val_tensor, right_arm_val_tensor, left_leg_val_tensor, right_leg_val_tensor, torso_val_tensor, reshaped_rnn_val_tensor, val_labels = batch

                output_spatial_val = spatial_net(left_arm_val_tensor, right_arm_val_tensor, left_leg_val_tensor,
                                                 right_leg_val_tensor, torso_val_tensor, edge_index)
                output_temporal_val = temporal_net(reshaped_rnn_val_tensor)
                concatenated_input_val = torch.cat((output_spatial_val, output_temporal_val), dim=1)
                final_output_val = final_model(concatenated_input_val)

                val_loss += criterion(final_output_val, val_labels).item()

                _, val_preds = torch.max(final_output_val, 1)
                all_val_preds.extend(val_preds.cpu().numpy())
                all_val_labels.extend(val_labels.cpu().numpy())

        # Calculate metrics for validation
        val_accuracy, val_sensitivity, val_specificity, val_f1 = calculate_metrics(all_val_labels, all_val_preds)
        val_metrics['accuracy'].append(val_accuracy)
        val_metrics['sensitivity'].append(val_sensitivity)
        val_metrics['specificity'].append(val_specificity)
        val_metrics['f1'].append(val_f1)

        print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation Sensitivity: {val_sensitivity:.4f}, Validation Specificity: {val_specificity:.4f}, Validation F1: {val_f1:.4f}")

        scheduler.step()
        early_stopping(val_loss, final_model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    # Return all five values: validation loss, accuracy, sensitivity, specificity, and F1
    final_val_accuracy = val_metrics['accuracy'][-1]
    final_val_sensitivity = val_metrics['sensitivity'][-1]
    final_val_specificity = val_metrics['specificity'][-1]
    final_val_f1 = val_metrics['f1'][-1]

    return val_loss, final_val_accuracy, final_val_sensitivity, final_val_specificity, final_val_f1

def cross_val_significance_test(dataset, labels, n_splits, batch_size, alpha, gamma, learning_rate, weight_decay,
                                edge_index, class_names, spatial_net, temporal_net, final_model, baseline=None):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    accuracies, sensitivities, specificities, f1_scores = [], [], [], []

    for fold, (train_index, test_index) in enumerate(skf.split(np.zeros(len(labels)), labels)):
        print(f"Processing fold {fold + 1}/{n_splits}...")

        train_loader = create_data_loader(dataset, train_index, batch_size)
        test_loader = create_data_loader(dataset, test_index, batch_size)

        val_loss, accuracy, sensitivity, specificity, f1 = train_model(
            alpha=alpha,
            gamma=gamma,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            train_loader=train_loader,
            val_loader=test_loader,
            edge_index=edge_index,
            class_names=class_names,
            spatial_net=spatial_net,
            temporal_net=temporal_net,
            final_model=final_model
        )
        accuracies.append(accuracy)
        sensitivities.append(sensitivity)
        specificities.append(specificity)
        f1_scores.append(f1)

        print(f"Fold {fold + 1}: Accuracy: {accuracy:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}, F1: {f1:.4f}")

    # Calculate mean values for each metric directly from stored values
    mean_accuracy = np.mean(accuracies)
    mean_sensitivity = np.mean(sensitivities)
    mean_specificity = np.mean(specificities)
    mean_f1 = np.mean(f1_scores)

    # Calculate confidence intervals for each metric
    ci_accuracy = bootstrap_confidence_interval(accuracies)
    ci_sensitivity = bootstrap_confidence_interval(sensitivities)
    ci_specificity = bootstrap_confidence_interval(specificities)
    ci_f1 = bootstrap_confidence_interval(f1_scores)

    # Calculate p-values if baseline is provided
    if baseline is not None:
        p_accuracy = ttest_1samp(accuracies, baseline).pvalue
        p_sensitivity = ttest_1samp(sensitivities, baseline).pvalue
        p_specificity = ttest_1samp(specificities, baseline).pvalue
        p_f1 = ttest_1samp(f1_scores, baseline).pvalue
    else:
        p_accuracy = p_sensitivity = p_specificity = p_f1 = None

    # Print results with the specified format
    print(f"Mean Accuracy: {mean_accuracy:.4f}, CI: {ci_accuracy}, p-value: {p_accuracy:.4f}" if p_accuracy is not None else f"Mean Accuracy: {mean_accuracy:.4f}, CI: {ci_accuracy}, p-value: None")
    print(f"Mean Sensitivity: {mean_sensitivity:.4f}, CI: {ci_sensitivity}, p-value: {p_sensitivity:.4f}" if p_sensitivity is not None else f"Mean Sensitivity: {mean_sensitivity:.4f}, CI: {ci_sensitivity}, p-value: None")
    print(f"Mean Specificity: {mean_specificity:.4f}, CI: {ci_specificity}, p-value: {p_specificity:.4f}" if p_specificity is not None else f"Mean Specificity: {mean_specificity:.4f}, CI: {ci_specificity}, p-value: None")
    print(f"Mean F1 Score: {mean_f1:.4f}, CI: {ci_f1}, p-value: {p_f1:.4f}" if p_f1 is not None else f"Mean F1 Score: {mean_f1:.4f}, CI: {ci_f1}, p-value: None")

    return (mean_accuracy, ci_accuracy, p_accuracy,
            mean_sensitivity, ci_sensitivity, p_sensitivity,
            mean_specificity, ci_specificity, p_specificity,
            mean_f1, ci_f1, p_f1)

# Train the models globally
spatial_net = SpatialNetwork(n_part, n_graph_out, frame_nos, leaky_relu_alpha)
temporal_net = TemporalNetwork(input_size, hidden_size, output_size, num_layers, dropout_prob, leaky_relu_alpha)
final_model = FinalConcatenation()

# Initialize the models
custom_initialize_weights(spatial_net)
initialize_weights(temporal_net)
initialize_weights(final_model)

# Example usage
dataset = TensorDataset(left_arm_train_tensor, right_arm_train_tensor, left_leg_train_tensor,
                        right_leg_train_tensor, torso_train_tensor, reshaped_rnn_train_tensor, train_labels)

mean_accuracy, ci_accuracy, p_accuracy, mean_sensitivity, ci_sensitivity, p_sensitivity, mean_specificity, ci_specificity, p_specificity, mean_f1, ci_f1, p_f1 = cross_val_significance_test(
    dataset=dataset,
    labels=train_labels,
    n_splits=5,
    batch_size=32,
    baseline=0.5,
    alpha=alpha,
    gamma=gamma,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    edge_index=edge_index,
    class_names=class_names,
    spatial_net=spatial_net,
    temporal_net=temporal_net,
    final_model=final_model
)

print(f"Mean Accuracy: {mean_accuracy:.4f}, CI: {ci_accuracy}, p-value: {p_accuracy:.4f}")
print(f"Mean Sensitivity: {mean_sensitivity:.4f}, CI: {ci_sensitivity}, p-value: {p_sensitivity:.4f}")
print(f"Mean Specificity: {mean_specificity:.4f}, CI: {ci_specificity}, p-value: {p_specificity:.4f}")
print(f"Mean F1 Score: {mean_f1:.4f}, CI: {ci_f1}, p-value: {p_f1:.4f}")


Processing fold 1/5...
Epoch [1/47], Training Loss: 0.035442691296339035, Training Accuracy: 0.5000, Training Sensitivity: 0.7660, Training Specificity: 0.3077, Training F1: 0.5625
Epoch [1/47], Validation Loss: 0.0826, Validation Accuracy: 0.4219, Validation Sensitivity: 0.8571, Validation Specificity: 0.0833, Validation F1: 0.5647
Validation loss decreased (inf --> 0.082588).  Saving model ...
Epoch [2/47], Training Loss: 0.056636422872543335, Training Accuracy: 0.4330, Training Sensitivity: 0.6316, Training Specificity: 0.2868, Training F1: 0.4858
Epoch [2/47], Validation Loss: 0.0783, Validation Accuracy: 0.4375, Validation Sensitivity: 0.9286, Validation Specificity: 0.0556, Validation F1: 0.5909
Validation loss decreased (0.082588 --> 0.078319).  Saving model ...
Epoch [3/47], Training Loss: 0.041823841631412506, Training Accuracy: 0.4955, Training Sensitivity: 0.6632, Training Specificity: 0.3721, Training F1: 0.5272
Epoch [3/47], Validation Loss: 0.0788, Validation Accuracy: 0.